## Import Datasets Notebook
-process for importing WSI or DCM datasets (images & labels) into Exact

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import pandas as pd

from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi

from exact_sync.v1.models import ImageSet, Team, Product, AnnotationType, Image, Annotation, AnnotationMediaFile
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

from pathlib import Path

### Set-up Exact Server Connection

In [3]:
configuration = Configuration()
configuration.username = 'bnapora'
configuration.password = 'bnapora'
configuration.host = "http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)

In [4]:
# image_sets_api.list_image_sets()

In [5]:
image_sets = image_sets_api.list_image_sets(name__contains='CCMCT')
image_sets


{'count': 1,
 'next': None,
 'previous': None,
 'results': [{'creator': 1,
              'description': '',
              'id': 4,
              'images': [55,
                         56,
                         57,
                         58,
                         59,
                         60,
                         61,
                         62,
                         63,
                         64,
                         65,
                         66,
                         67,
                         68,
                         69,
                         70,
                         71,
                         72,
                         73,
                         74,
                         75,
                         76,
                         77,
                         78,
                         79,
                         80,
                         81,
                         82,
                         83,
                         84,

In [6]:
list_imageset_images = []
image_sets.count
for image_set in image_sets.results:
    imageset_images = images_api.list_images(image_set=image_set.id)
    for image in imageset_images.results:
        list_imageset_images.append([image.id, image.filename])
#         list_imageset_images[image.filename] = image.id
        
print(list_imageset_images)

        

[[55, '2f17d43b3f9e7dacf24c.svs'], [56, '20c0753af38303691b27.svs'], [57, '285f74bb6be025a676b6.svs'], [58, 'c3eb4b8382b470dd63a9.svs'], [59, 'c91a842257ed2add5134.svs'], [60, 'f26e9fcef24609b988be.svs'], [61, '066c94c4c161224077a9.svs'], [62, '0e56fd11a762be0983f0.svs'], [63, '1018715d369dd0df2fc0.svs'], [64, '2e611073cff18d503cea.svs'], [65, '2efb541724b5c017c503.svs'], [66, '2f2591b840e83a4b4358.svs'], [67, '34eb28ce68c1106b2bac.svs'], [68, '39ecf7f94ed96824405d.svs'], [69, '3f2e034c75840cb901e6.svs'], [70, '552c51bfb88fd3e65ffe.svs'], [71, '70ed18cd5f806cf396f0.svs'], [72, '8bebdd1f04140ed89426.svs'], [73, '8c9f9618fcaca747b7c3.svs'], [74, '91a8e57ea1f9cb0aeb63.svs'], [75, '9374efe6ac06388cc877.svs'], [76, '96274538c93980aad8d6.svs'], [77, 'a0c8b612fe0655eab3ce.svs'], [78, 'ac1168b2c893d2acad38.svs'], [79, 'add0a9bbc53d1d9bac4c.svs'], [80, 'be10fa37ad6e88e1f406.svs'], [81, 'c86cd41f96331adf3856.svs'], [82, 'ce949341ba99845813ac.svs'], [83, 'dd4246ab756f6479c841.svs'], [84, 'dd6dd0d

### Upload Multiple Images from Directory

In [7]:
path_images = Path('/host_Data/DataSets/MITOS_WSI_CCMCT/WSI')
id_imageset = 4
type_wsi = {'.svs', '.scn'}
# [x for x in path_images.iterdir()]

In [8]:
# for path_image in path_images.iterdir():
#     if Path(path_image).suffix in type_wsi:
#         print (path_image)
#         images_api.create_image(file_path=path_image, image_set=id_imageset)

### Upload Annotations for Images

#### Import from SlideRunner (sqlite db)

In [9]:
pathDB = Path('/host_Data/DataSets/MITOS_WSI_CCMCT/databases')
dbname = 'MITOS_WSI_CCMCT_ODAEL.sqlite'

In [10]:
import sqlite3
from SlideRunner.dataAccess.database import Database
from tqdm import tqdm
import uuid

database = Database()
print(str(pathDB/dbname))
database.open(str(pathDB/dbname))

/host_Data/DataSets/MITOS_WSI_CCMCT/databases/MITOS_WSI_CCMCT_ODAEL.sqlite


In [12]:
# Create Class dictionary for Data Source
query_classes = 'SELECT uid, name from Classes'
classes_source = {}
for uid_class, name_class in database.execute(query_classes).fetchall():
    classes_source[uid_class] = name_class
print(classes_source)


{1: 'granulocyte', 2: 'mitotic figure', 3: 'tumor cell', 4: 'other/ambiguous cells', 5: 'binucleated cell', 6: 'multinukleated cell', 7: 'Mitotic figure lookalike', 11: 'background'}


In [13]:
# Create Class dictionary for Data Target (Exact)
annotation_types = annotation_types_api.list_annotation_types(product=3)
classes_exact = {}
for result in annotation_types.results:
    classes_exact[result.name] = result.id
print(classes_exact)

{'granulocyte': 13, 'mitotic figure': 14, 'tumor cell': 15, 'other/ambiguous cells': 16, 'binucleated cell': 17, 'multinukleated cell': 18, 'Mitotic figure lookalike': 19, 'background': 20}


In [14]:
#Map Class Dictionaries
classes_map = {}
for class_source in classes_source:
    classes_map[class_source] = classes_exact[classes_source[class_source]]
print(classes_map)

{1: 13, 2: 14, 3: 15, 4: 16, 5: 17, 6: 18, 7: 19, 11: 20}


In [15]:
radius = 25

In [127]:
results_annotations = []
for exact_image_id, filename in list_imageset_images[1:]:
    query_slides = 'SELECT uid, filename from Slides where filename = "' + filename + '"'
    for slide_id, filename in database.execute(query_slides).fetchall():
        database.loadIntoMemory(slide_id)
        for id, annotation in database.annotations.items():
            vector = {"x1": int(annotation.x1-radius), "y1": int(annotation.y1-radius), "x2": int(annotation.x1+radius), "y2": int(annotation.y1+radius)}
            label_name = classes_source[annotation.agreedClass]
            label_id = annotation.agreedClass
            results_annotations.append([exact_image_id, vector, label_id, label_name, slide_id])

,exact_image_id,vector,label_id,label_name,sliderunner_image_id
0,56,"{'x1': 59727, 'y1': 439, 'x2': 59777, 'y2': 489}",2,mitotic figure,21
1,56,"{'x1': 42473, 'y1': 7024, 'x2': 42523, 'y2': 7...",3,tumor cell,21
2,56,"{'x1': 43366, 'y1': 6888, 'x2': 43416, 'y2': 6...",3,tumor cell,21
3,56,"{'x1': 41866, 'y1': 7808, 'x2': 41916, 'y2': 7...",2,mitotic figure,21
4,56,"{'x1': 44239, 'y1': 7317, 'x2': 44289, 'y2': 7...",3,tumor cell,21


In [ ]:
pd_results_annotations = pd.DataFrame(results_annotations, columns=['exact_image_id', 'vector', 'label_id', 'label_name', 'sliderunner_image_id'])
pd_results_annotations.head()

In [128]:
len(pd_results_annotations)

254184

In [129]:
# pd_results_annotations_tmp = pd_results_annotations.iloc[0:3].copy()
# pd_results_annotations_tmp.head()

In [130]:
# help(annotations_api.create_annotation)

In [ ]:
threads = []
for image_id, vector, label in zip(pd_results_annotations['exact_image_id'], pd_results_annotations['vector'], pd_results_annotations['label_id']):
# for image_id, vector, label in zip(pd_results_annotations_tmp['exact_image_id'], pd_results_annotations_tmp['vector'], pd_results_annotations_tmp['label_id']):
    vector=vector
    unique_identifier = str(uuid.uuid4())
    exact_annotation_type = int(classes_map[label])
    image_id = int(image_id)
    annotation = Annotation(annotation_type=exact_annotation_type, image=image_id, vector=vector, unique_identifier=unique_identifier)
    thread = annotations_api.create_annotation(body=annotation)
    threads.append(thread)

In [126]:
annotation_results = annotations_api.list_annotations(image=55).results
print('Count of Annotations=', len(annotation_results))
# annotation_results

Count of Annotations= 50


[{'annotation_type': 13,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 404643,
  'image': 55,
  'last_edit_time': datetime.datetime(2021, 8, 17, 1, 43, 48, 919255),
  'last_editor': 1,
  'meta_data': None,
  'time': datetime.datetime(2021, 8, 17, 1, 43, 48, 919204),
  'unique_identifier': 'e93d5d19-36cb-4ea5-874e-cb9ed5628ed1',
  'uploaded_media_files': [],
  'user': 1,
  'vector': {'x1': 21084, 'x2': 21134, 'y1': 10267, 'y2': 10317},
  'verified_by_user': 'False'},
 {'annotation_type': 13,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 404644,
  'image': 55,
  'last_edit_time': datetime.datetime(2021, 8, 17, 1, 43, 49, 57036),
  'last_editor': 1,
  'meta_data': None,
  'time': datetime.datetime(2021, 8, 17, 1, 43, 49, 56979),
  'unique_identifier': 'f8ac135a-89bb-4590-93e6-67923464cc4c',
  'uploaded_media_files': [],
  'user': 1

In [121]:
#delete annotations
for anno in annotation_results:
    annotations_api.destroy_annotation(id=anno.id)
    print(anno.id)
    

404640
404641
404642


In [119]:
while (len(threads) > 0):
    for thread in threads:
        if thread.ready():
            data.results += thread.get().results
            threads.remove(thread)
    sleep(0.25)

AttributeError: 'Annotation' object has no attribute 'ready'